### To calculate the mAP first ineed to get the  OKS for each keypoint with its corresponding prediction

#### imports and installs

In [1]:
!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/f8/3e/021fe648b56598e3e5267caf418dc593d70e4ea44922c60e9b9775474da6/ultralytics-8.0.167-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.2/613.2 kB 14.7 MB/s eta 0:00:0000:01


In [2]:
import os
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch

In [3]:
from IPython.display import clear_output
!pip install labelbox[data]
import labelbox
from s3 import get_image_dimensions_from_s3_bucket
from PIL import Image  
from io import BytesIO,StringIO
from tqdm import tqdm  
from datetime import datetime 
from requests.exceptions import RequestException 
import boto3
clear_output()

## Getting the Predicted and GroundTruth values of key points and bonding boxes

giving the paths for model to choose and image to be predicted

In [4]:
cwd = os.getcwd()
cwd

'/home/ec2-user/SageMaker'

In [5]:
image_id='25debad9ccacdc856d29b149e5802415330870e4c9f61489bfc1b1f88a5c5f41'

In [6]:
model_path=cwd+'/runs/pose/train17/weights/best.pt'
image_path=cwd+'/dataset_1/images/val/'+image_id+'.jpg'
label_path=cwd+'/dataset_1/labels/val/'+image_id+'.txt'

In [7]:
#importng the model
model = YOLO(model_path)

In [8]:
#results on validation set
val_results=model.val()

Ultralytics YOLOv8.0.167 🚀 Python-3.10.12 torch-2.0.1 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-pose summary (fused): 287 layers, 69464391 parameters, 0 gradients
100%|██████████| 755k/755k [00:00<00:00, 53.4MB/s]
val: Scanning /home/ec2-user/SageMaker/dataset_1/labels/val3.cache... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:05<00:00,  5.64s/it]
                   all          1          2          1          1      0.995      0.958        0.5        0.5      0.622      0.498
Speed: 0.4ms preprocess, 103.0ms inference, 0.0ms loss, 594.6ms postprocess per image
Results saved to runs/pose/val36


In [9]:
#running the model on the image and getting predictions
results=model(image_path)


image 1/1 /home/ec2-user/SageMaker/dataset_1/images/val/25debad9ccacdc856d29b149e5802415330870e4c9f61489bfc1b1f88a5c5f41.jpg: 480x640 2 fuse_blocks, 67.2ms
Speed: 1.9ms preprocess, 67.2ms inference, 21.6ms postprocess per image at shape (1, 3, 480, 640)


In [10]:
len(results)

1

#### Prdeicted bounding boxes

In [11]:
results[0].boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[412.0000, 591.0000, 567.0000, 687.0000,   0.8801,   0.0000],
        [404.0000, 465.0000, 561.0000, 564.0000,   0.8712,   0.0000]], device='cuda:0')
cls: tensor([0., 0.], device='cuda:0')
conf: tensor([0.8801, 0.8712], device='cuda:0')
data: tensor([[412.0000, 591.0000, 567.0000, 687.0000,   0.8801,   0.0000],
        [404.0000, 465.0000, 561.0000, 564.0000,   0.8712,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (960, 1280)
shape: torch.Size([2, 6])
xywh: tensor([[489.5000, 639.0000, 155.0000,  96.0000],
        [482.5000, 514.5000, 157.0000,  99.0000]], device='cuda:0')
xywhn: tensor([[0.3824, 0.6656, 0.1211, 0.1000],
        [0.3770, 0.5359, 0.1227, 0.1031]], device='cuda:0')
xyxy: tensor([[412., 591., 567., 687.],
        [404., 465., 561., 564.]], device='cuda:0')
xyxyn: tensor([[0.3219, 0.6156, 0.4430, 0.7156],
        [0.3156, 0.4844, 0.4383, 0.5875]], device='cuda:0')

In [12]:
predicted_bbx=results[0].boxes.xywh
print(predicted_bbx.shape)
predicted_bbx

torch.Size([2, 4])


tensor([[489.5000, 639.0000, 155.0000,  96.0000],
        [482.5000, 514.5000, 157.0000,  99.0000]], device='cuda:0')

#### Predicted Keypoints

In [13]:
results[0].keypoints

ultralytics.engine.results.Keypoints object with attributes:

conf: tensor([[9.1569e-01, 9.4305e-01, 9.4718e-01, 9.5455e-01, 9.4256e-01, 9.3398e-01, 9.2955e-01, 8.8639e-01, 1.7828e-03, 3.8250e-04, 9.1337e-01, 9.1913e-01, 9.3527e-01, 9.4529e-01, 9.4643e-01, 9.3883e-01, 9.2764e-01, 9.0341e-01, 4.2924e-03, 9.1525e-04],
        [9.4259e-01, 9.6104e-01, 9.6349e-01, 9.6903e-01, 9.5533e-01, 9.4561e-01, 9.3676e-01, 8.9347e-01, 1.5317e-03, 3.8471e-04, 9.4519e-01, 9.5694e-01, 9.6125e-01, 9.6927e-01, 9.6588e-01, 9.5943e-01, 9.4512e-01, 9.2387e-01, 4.4853e-03, 8.9082e-04]], device='cuda:0')
data: tensor([[[4.2930e+02, 6.7554e+02, 9.1569e-01],
         [4.4761e+02, 6.7602e+02, 9.4305e-01],
         [4.6354e+02, 6.7579e+02, 9.4718e-01],
         [4.8270e+02, 6.7639e+02, 9.5455e-01],
         [4.9646e+02, 6.7550e+02, 9.4256e-01],
         [5.1665e+02, 6.7575e+02, 9.3398e-01],
         [5.3167e+02, 6.7497e+02, 9.2955e-01],
         [5.4854e+02, 6.7485e+02, 8.8639e-01],
         [5.1577e+02, 6.7641e+02

In [14]:

a=results[0].keypoints
required=a.xy
predicted_keypoints=required.cpu()
#predicted_keypoints=required.numpy()
print(predicted_keypoints.shape)
predicted_keypoints

torch.Size([2, 20, 2])


tensor([[[429.3005, 675.5417],
         [447.6104, 676.0213],
         [463.5447, 675.7867],
         [482.7044, 676.3920],
         [496.4608, 675.5003],
         [516.6529, 675.7534],
         [531.6736, 674.9695],
         [548.5378, 674.8505],
         [515.7711, 676.4062],
         [487.2181, 642.3475],
         [426.3239, 602.1943],
         [443.4270, 601.2421],
         [461.7065, 601.9921],
         [480.9495, 602.0515],
         [492.0370, 601.1072],
         [513.1121, 602.0702],
         [529.0063, 601.4415],
         [547.6006, 601.6218],
         [547.4628, 674.7498],
         [533.1843, 673.2497]],

        [[424.4624, 552.8176],
         [442.1826, 553.1485],
         [458.6535, 552.5541],
         [478.2027, 552.4981],
         [491.3026, 551.1726],
         [511.9982, 551.4327],
         [527.2424, 550.9009],
         [543.7236, 550.4636],
         [512.5330, 553.5851],
         [472.4994, 510.7437],
         [419.3019, 479.3188],
         [437.1511, 478.0549],
      

#### Extracting ground truth values

In [15]:
def read_txt_to_list(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Removing newline characters and any leading/trailing spaces
            content_list = [line.strip() for line in lines]
            return content_list
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return []


labels = read_txt_to_list(label_path)
labels

['0 0.378125 0.5390625 0.121875 0.096875 0.33340390625 0.5742635416666667 2 0.3467828125 0.5750447916666667 2 0.35938046875 0.5764770833333334 2 0.37129453125 0.5737427083333333 2 0.38555234375 0.5766072916666667 2 0.396978125 0.574784375 2 0.41133359374999995 0.5725708333333334 2 0.42393125 0.5758260416666666 2 0 0 0 0 0 0 0.3256890625 0.5005229166666667 2 0.339946875 0.4981791666666667 2 0.35391171875 0.4997416666666667 2 0.367290625 0.4989604166666667 2 0.38018125 0.4977885416666667 2 0.39414609375 0.49726770833333334 2 0.40791562499999995 0.4984395833333333 2 0.42070859374999997 0.5001322916666667 2 0 0 0 0 0 0',
 '0 0.381640625 0.6640625 0.11953125 0.10104166666666667 0.335259375 0.7038645833333333 2 0.34736875 0.7024760416666667 2 0.3622125 0.7040385416666666 2 0.37341015624999996 0.7047322916666666 2 0.38578046875 0.7061218749999999 2 0.399321875 0.7054270833333334 2 0.41104062499999994 0.70803125 2 0.42731640625 0.7064687500000001 2 0 0 0 0 0 0 0.33340390625 0.6222239583333333 

In [16]:
im = Image.open(image_path)
w,h=im.size
print(w,h)

1280 960


In [17]:
def get_GT_values(labels,w,h):
    data=[]
    bbx=[]
    for i in labels:
        k=[]
        for t in i.split(' '):
            k.append(float(t))
        j = k[5:]
        #print(k[1:5])
        bb=[k[s]*h if s%2==0 else k[s]*w for s in range(1,5)]#keypoints are the form of x, y x should be multiplied by width and y with height
        data.append(j)
        #print(bb)
        bbx.append(bb)
    GT_keypoints=[]
    for i in data:
        l=[]
        for k in range(0,len(i)-2,3):
            t=[]
            t.append(i[k]*w)
            t.append(i[k+1]*h)
            l.append(t)
        GT_keypoints.append(l)
    #keypoints are the form of x, y x should be multiplied by width and y with height
    
    return torch.tensor(bbx),torch.tensor(GT_keypoints)


GT_bbx,GT_keypoints= get_GT_values(labels,w,h)

In [18]:
print(GT_bbx.shape)
GT_bbx

torch.Size([2, 4])


tensor([[484.0000, 517.5000, 156.0000,  93.0000],
        [488.5000, 637.5000, 153.0000,  97.0000]])

In [19]:
print(GT_keypoints.shape)
GT_keypoints

torch.Size([2, 20, 2])


tensor([[[426.7570, 551.2930],
         [443.8820, 552.0430],
         [460.0070, 553.4180],
         [475.2570, 550.7930],
         [493.5070, 553.5430],
         [508.1320, 551.7930],
         [526.5070, 549.6680],
         [542.6320, 552.7930],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [416.8820, 480.5020],
         [435.1320, 478.2520],
         [453.0070, 479.7520],
         [470.1320, 479.0020],
         [486.6320, 477.8770],
         [504.5070, 477.3770],
         [522.1320, 478.5020],
         [538.5070, 480.1270],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000]],

        [[429.1320, 675.7100],
         [444.6320, 674.3770],
         [463.6320, 675.8770],
         [477.9650, 676.5430],
         [493.7990, 677.8770],
         [511.1320, 677.2100],
         [526.1320, 679.7100],
         [546.9650, 678.2100],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [426.7570, 597.3350],
         [441.3820, 598.8350],
      

#### Extracting the area of the groudtruth bboxes

In [20]:
area=[]
for i in GT_bbx:
    a=i[2]*i[3]
    area.append(a)
area = torch.tensor(area)
area

tensor([14508., 14841.])

#### Sigma . sigma (list): A list containing 17 values representing keypoint scales.

In [21]:
sigma=[1/20 for i in GT_keypoints[0]]
sigma=np.array(sigma) 

len(sigma)
sigma

array([       0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05,        0.05])

## OKS calculation 

In [22]:
def kpt_iou(kpt1, kpt2, area, sigma, eps=1e-7):
    """
    Calculate Object Keypoint Similarity (OKS).

    Args:
        kpt1 (torch.Tensor): A tensor of shape (N, 17, 3) representing ground truth keypoints.
        kpt2 (torch.Tensor): A tensor of shape (M, 17, 3) representing predicted keypoints.
        area (torch.Tensor): A tensor of shape (N,) representing areas from ground truth.
        sigma (list): A list containing 17 values representing keypoint scales.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-7.

    Returns:
        (torch.Tensor): A tensor of shape (N, M) representing keypoint similarities.
    """
    d = (kpt1[:, None, :, 0] - kpt2[..., 0]) ** 2 + (kpt1[:, None, :, 1] - kpt2[..., 1]) ** 2  # (N, M, 17)
    sigma = torch.tensor(sigma, device=kpt1.device, dtype=kpt1.dtype)  # (17, )
    
    kpt_mask = kpt1[..., 1] != 0  # (N, 17)

    e = d / (2 * sigma) ** 2 / (area[:, None, None] + eps) / 2  # from cocoeval
    #print(f'e: {e}, shape: {e.shape}')
    # e = d / ((area[None, :, None] + eps) * sigma) ** 2 / 2  # from formula
    
    
    return (torch.exp(-e) * kpt_mask[:, None]).sum(-1) / (kpt_mask.sum(-1)[:, None] + eps), d

In [23]:
oks,d=kpt_iou(GT_keypoints, predicted_keypoints, area, sigma, eps=1e-7)
oks

tensor([[2.3910e-23, 9.6732e-01],
        [9.3315e-01, 3.9905e-22]])

In [24]:
d1=d.tolist()
d2=[d1[0][0],d1[1][1]]
ed=[]
for i in d2:
    for j in i:
        ed.append(j)
ed

[15444.1982421875,
 15384.51171875,
 14986.6171875,
 15830.572265625,
 14882.3017578125,
 15438.7685546875,
 15727.1640625,
 14932.8955078125,
 723545.1875,
 649991.75,
 14898.169921875,
 15195.3603515625,
 15018.3115234375,
 15258.1826171875,
 15214.90234375,
 15622.4365234375,
 15161.365234375,
 14843.6826171875,
 755002.6875,
 737550.625,
 15124.345703125,
 14702.3525390625,
 15233.33203125,
 15387.19921875,
 16060.23828125,
 15820.6748046875,
 16593.02734375,
 16329.6494140625,
 569146.625,
 484114.875,
 13983.40234375,
 14605.73046875,
 14258.4560546875,
 14600.8994140625,
 15757.48046875,
 15384.150390625,
 15088.66796875,
 15083.9033203125,
 598833.9375,
 582240.3125]

#### OKS calculation for GT to corresponding prediction keypoints

In [25]:
def kpt_iou2(kpt1, kpt2, area, sigma, eps=1e-7):
    """
    Calculate Object Keypoint Similarity (OKS).

    Args:
        kpt1 (torch.Tensor): A tensor of shape (N, 17, 3) representing ground truth keypoints.
        kpt2 (torch.Tensor): A tensor of shape (M, 17, 3) representing predicted keypoints.
        area (torch.Tensor): A tensor of shape (N,) representing areas from ground truth.
        sigma (list): A list containing 17 values representing keypoint scales.
        eps (float, optional): A small value to avoid division by zero. Defaults to 1e-7.

    Returns:
        (torch.Tensor): A tensor of shape (N, M) representing keypoint similarities.
    """
    d = (kpt1[:, None, :, 0] - kpt2[..., 0]) ** 2 + (kpt1[:, None, :, 1] - kpt2[..., 1]) ** 2  # (N, M, 17)
    sigma = torch.tensor(sigma, device=kpt1.device, dtype=kpt1.dtype)  # (17, )
    kpt_mask = kpt1[..., 1] != 0  # (N, 17)
    e = d / (2 * sigma) ** 2 / (area[:, None, None] + eps) / 2  # from cocoeval
    #print(f'e: {e}, shape: {e.shape}')
    # e = d / ((area[None, :, None] + eps) * sigma) ** 2 / 2  # from formula
    
    
    return (torch.exp(-e) * kpt_mask[:, None])/ (kpt_mask[:, None] + eps)

In [26]:
OKS=kpt_iou2(GT_keypoints, predicted_keypoints, area, sigma, eps=1e-7)
OKS

tensor([[[7.6565e-24, 9.4051e-24, 3.7061e-23, 2.0218e-24, 5.3094e-23, 7.8011e-24, 2.8874e-24, 4.4598e-23, 0.0000e+00, 0.0000e+00, 5.0268e-23, 1.8050e-23, 3.3226e-23, 1.4536e-23, 1.6874e-23, 4.1424e-24, 2.0294e-23, 6.0652e-23, 0.0000e+00, 0.0000e+00],
         [9.7418e-01, 9.8594e-01, 9.9115e-01, 9.6086e-01, 9.6453e-01, 9.4936e-01, 9.9292e-01, 9.7745e-01, 0.0000e+00, 0.0000e+00, 9.7530e-01, 9.8592e-01, 9.6724e-01, 8.9518e-01, 9.9916e-01, 9.5437e-01, 9.7901e-01, 9.2456e-01, 0.0000e+00, 0.0000e+00]],

        [[9.9981e-01, 9.6176e-01, 9.9995e-01, 9.2705e-01, 9.5801e-01, 8.9598e-01, 8.3596e-01, 9.5470e-01, 0.0000e+00, 0.0000e+00, 9.2295e-01, 9.6695e-01, 8.8896e-01, 8.7452e-01, 9.8797e-01, 9.2136e-01, 9.3482e-01, 8.9967e-01, 0.0000e+00, 0.0000e+00],
         [7.4250e-23, 3.0771e-22, 5.1432e-23, 3.0627e-23, 3.1720e-24, 7.1098e-24, 5.2697e-25, 1.2798e-24, 0.0000e+00, 0.0000e+00, 3.4680e-21, 4.2610e-22, 1.3729e-21, 4.3309e-22, 8.7968e-24, 3.0943e-23, 8.3733e-23, 8.5089e-23, 0.0000e+00, 0.0000e

### Claculating mAP score 

ap_per_class function Computes the average precision per class for object detection evaluation.

arguments needed for this function are :
1) tp np.ndarray): True positive counts for each class.
2) conf (np.ndarray): Array of confidence scores of the detections.
3) pred_cls (np.ndarray): Array of predicted classes of the detections.
4) target_cls (np.ndarray): Array of true classes of the detections.
5) plot (bool, optional): Whether to plot PR curves or not. Defaults to False.
6) on_plot (func, optional): A callback to pass plots path and data when they are rendered. Defaults to None.
7) save_dir (Path, optional): Directory to save the PR curves. Defaults to an empty path.
8) names (tuple, optional): Tuple of class names to plot PR curves. Defaults to an empty tuple.
9) eps (float, optional): A small value to avoid division by zero. Defaults to 1e-16.
10) prefix (str, optional): A prefix string for saving the plot files. Defaults to an empty string.


In [27]:
def ap_per_class(tp,
                 conf,
                 pred_cls,
                 target_cls,
                 plot=False,
                 on_plot=None,
                 save_dir=Path(),
                 names={},
                 eps=1e-16,
                 prefix=''):
    """

    Returns:
        (tuple): A tuple of six arrays and one array of unique classes, where:
            tp (np.ndarray): True positive counts for each class.
            fp (np.ndarray): False positive counts for each class.
            p (np.ndarray): Precision values at each confidence threshold.
            r (np.ndarray): Recall values at each confidence threshold.
            f1 (np.ndarray): F1-score values at each confidence threshold.
            ap (np.ndarray): Average precision for each class at different IoU thresholds.
            unique_classes (np.ndarray): An array of unique classes that have data.

    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes, nt = np.unique(target_cls, return_counts=True)
    nc = unique_classes.shape[0]  # number of classes, number of detections

    # Create Precision-Recall curve and compute AP for each class
    px, py = np.linspace(0, 1, 1000), []  # for plotting
    ap, p, r = np.zeros((nc, tp.shape[1])), np.zeros((nc, 1000)), np.zeros((nc, 1000))
    for ci, c in enumerate(unique_classes):
        i = pred_cls == c
        n_l = nt[ci]  # number of labels
        n_p = i.sum()  # number of predictions
        if n_p == 0 or n_l == 0:
            continue

        # Accumulate FPs and TPs
        fpc = (1 - tp[i]).cumsum(0)
        tpc = tp[i].cumsum(0)

        # Recall
        recall = tpc / (n_l + eps)  # recall curve
        r[ci] = np.interp(-px, -conf[i], recall[:, 0], left=0)  # negative x, xp because xp decreases
        print("RECALL")
        print(recall)
        print("___________________________________")
        # Precision
        precision = tpc / (tpc + fpc)  # precision curve
        p[ci] = np.interp(-px, -conf[i], precision[:, 0], left=1)  # p at pr_score
        print("PRECISION")
        print(precision)
        print("___________________________________")

        # AP from recall-precision curve
        for j in range(tp.shape[1]):
            ap[ci, j], mpre, mrec = compute_ap(recall[:, j], precision[:, j])
            if plot and j == 0:
                py.append(np.interp(px, mrec, mpre))  # precision at mAP@0.5

    # Compute F1 (harmonic mean of precision and recall)
    f1 = 2 * p * r / (p + r + eps)
    names = [v for k, v in names.items() if k in unique_classes]  # list: only classes that have data
    names = dict(enumerate(names))  # to dict
    '''if plot:
        plot_pr_curve(px, py, ap, save_dir / f'{prefix}PR_curve.png', names, on_plot=on_plot)
        plot_mc_curve(px, f1, save_dir / f'{prefix}F1_curve.png', names, ylabel='F1', on_plot=on_plot)
        plot_mc_curve(px, p, save_dir / f'{prefix}P_curve.png', names, ylabel='Precision', on_plot=on_plot)
        plot_mc_curve(px, r, save_dir / f'{prefix}R_curve.png', names, ylabel='Recall', on_plot=on_plot)'''

    i = smooth(f1.mean(0), 0.1).argmax()  # max F1 index
    p, r, f1 = p[:, i], r[:, i], f1[:, i]
    tp = (r * nt).round()  # true positives
    fp = (tp / (p + eps) - tp).round()  # false positives
    return tp, fp, p, r, f1, ap, unique_classes.astype(int)


##### tp (np.ndarray): True positive counts for each class, Calculating true positives in our case

In [28]:
oks_array=np.array([OKS[0][1].tolist(),OKS[1][0].tolist()])
print(oks_array.shape)
oks_array

(2, 20)


array([[    0.97418,     0.98594,     0.99115,     0.96086,     0.96453,     0.94936,     0.99292,     0.97745,           0,           0,      0.9753,     0.98592,     0.96724,     0.89518,     0.99916,     0.95437,     0.97901,     0.92456,           0,           0],
       [    0.99981,     0.96176,     0.99995,     0.92705,     0.95801,     0.89598,     0.83596,      0.9547,           0,           0,     0.92295,     0.96695,     0.88896,     0.87452,     0.98797,     0.92136,     0.93482,     0.89967,           0,           0]])

In [29]:
import numpy as np
oks_binary=[]
for i in oks_array:
    #print(i)
    l=[]
    for j in i:
        #print(j)
        if j>=0.9:
                l.append(1)
        else:
                l.append(0)
    oks_binary.append(l)

'''tp=np.asarray(oks_binary)
print(tp.shape)'''
tp= np.asarray([item for sublist in oks_binary for item in sublist])
tp

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0])

In [30]:
tp = tp.reshape((len(tp), 1))

In [31]:
tp.shape

(40, 1)

#### Confidence score- conf (np.ndarray): Array of confidence scores of the detections.

In [32]:
conf_tensor=results[0].keypoints.conf
#conf=np.asarray(conf_tensor.tolist())
conf=conf_tensor.tolist()
#print(conf.shape)
conf

[[0.9156899452209473,
  0.943047046661377,
  0.9471763968467712,
  0.9545457363128662,
  0.9425613880157471,
  0.9339821338653564,
  0.9295464158058167,
  0.8863851428031921,
  0.0017828426789492369,
  0.0003824982268270105,
  0.9133736491203308,
  0.9191308617591858,
  0.9352748990058899,
  0.9452911019325256,
  0.9464324116706848,
  0.9388270974159241,
  0.9276393055915833,
  0.9034082293510437,
  0.004292359575629234,
  0.0009152456768788397],
 [0.9425906538963318,
  0.9610418081283569,
  0.9634930491447449,
  0.9690299034118652,
  0.955328643321991,
  0.9456093311309814,
  0.9367578029632568,
  0.8934694528579712,
  0.0015317475190386176,
  0.000384712329832837,
  0.9451879262924194,
  0.956940770149231,
  0.9612489342689514,
  0.9692705273628235,
  0.9658834338188171,
  0.9594281911849976,
  0.9451246857643127,
  0.923872172832489,
  0.004485331941395998,
  0.0008908185991458595]]

In [33]:
conf=np.asarray([item for sublist in conf for item in sublist])
conf

array([    0.91569,     0.94305,     0.94718,     0.95455,     0.94256,     0.93398,     0.92955,     0.88639,   0.0017828,   0.0003825,     0.91337,     0.91913,     0.93527,     0.94529,     0.94643,     0.93883,     0.92764,     0.90341,   0.0042924,  0.00091525,     0.94259,     0.96104,     0.96349,     0.96903,
           0.95533,     0.94561,     0.93676,     0.89347,   0.0015317,  0.00038471,     0.94519,     0.95694,     0.96125,     0.96927,     0.96588,     0.95943,     0.94512,     0.92387,   0.0044853,  0.00089082])

In [34]:
conf.shape

(40,)

#### Pred_cls - pred_cls (np.ndarray): Array of predicted classes of the detections.

In [35]:
''' here we were not sure of what pred_cls means in this context'''

' here we were not sure of what pred_cls means in this context'

In [36]:
pred_cls=np.asarray([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19, 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])
target_cls=pred_cls
print(pred_cls.shape,target_cls.shape)

(40,) (40,)


In [37]:
l=[8,9,18,19,28,29,38,39]
pred_cls2=np.asarray([1 if i not in l else 0 for i in range(40)])
target_cls2=pred_cls2
print(pred_cls2.shape,target_cls2.shape)
print(pred_cls2)

(40,) (40,)
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 0]


In [38]:
l=[8,9,18,19,28,29,38,39]
target_cls2=np.asarray([2 if i not in l else 0 for i in range(40)])
pred_cls2=np.asarray([2  for i in range(40)])
print(pred_cls2.shape,target_cls2.shape)
print(pred_cls2)

(40,) (40,)
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [39]:
pred_cls2=np.asarray([ 0 for i in range(40)])
target_cls2=pred_cls2
print(pred_cls2.shape,target_cls2.shape)
print(pred_cls2)

(40,) (40,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [40]:
def compute_ap(recall, precision):
    """
    Compute the average precision (AP) given the recall and precision curves.

    Arguments:
        recall (list): The recall curve.
        precision (list): The precision curve.

    Returns:
        (float): Average precision.
        (np.ndarray): Precision envelope curve.
        (np.ndarray): Modified recall curve with sentinel values added at the beginning and end.
    """

    # Append sentinel values to beginning and end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([1.0], precision, [0.0]))

    # Compute the precision envelope
    mpre = np.flip(np.maximum.accumulate(np.flip(mpre)))

    # Integrate area under curve
    method = 'interp'  # methods: 'continuous', 'interp'
    if method == 'interp':
        x = np.linspace(0, 1, 101)  # 101-point interp (COCO)
        ap = np.trapz(np.interp(x, mrec, mpre), x)  # integrate
    else:  # 'continuous'
        i = np.where(mrec[1:] != mrec[:-1])[0]  # points where x-axis (recall) changes
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])  # area under curve

    return ap, mpre, mrec

In [41]:
def smooth(y, f=0.05):
    """Box filter of fraction f."""
    nf = round(len(y) * f * 2) // 2 + 1  # number of filter elements (must be odd)
    p = np.ones(nf // 2)  # ones padding
    yp = np.concatenate((p * y[0], y, p * y[-1]), 0)  # y padded
    return np.convolve(yp, np.ones(nf) / nf, mode='valid')  # y-smoothed

In [42]:
ap_results2=ap_per_class(tp,
                 conf,
                 pred_cls2,
                 target_cls2,
                 save_dir='/home/ec2-user/SageMaker/trial2',
                 eps=1e-16,
                 prefix='Pose')
ap_results2

RECALL
[[          0]
 [      0.025]
 [       0.05]
 [      0.075]
 [      0.075]
 [        0.1]
 [      0.125]
 [       0.15]
 [      0.175]
 [        0.2]
 [      0.225]
 [       0.25]
 [       0.25]
 [       0.25]
 [      0.275]
 [        0.3]
 [      0.325]
 [       0.35]
 [      0.375]
 [        0.4]
 [        0.4]
 [      0.425]
 [       0.45]
 [      0.475]
 [        0.5]
 [        0.5]
 [      0.525]
 [       0.55]
 [      0.575]
 [        0.6]
 [      0.625]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]
 [       0.65]]
___________________________________
PRECISION
[[          0]
 [        0.5]
 [    0.66667]
 [       0.75]
 [        0.6]
 [    0.66667]
 [    0.71429]
 [       0.75]
 [    0.77778]
 [        0.8]
 [    0.81818]
 [    0.83333]
 [    0.76923]
 [    0.71429]
 [    0.73333]
 [       0.75]
 [    0.76471]
 [    0.77778]
 [    0.78947]
 [        0.8]
 [     0.7619]
 [    0.77273]
 [    0.78261]
 

(array([         26]),
 array([          6]),
 array([    0.81109]),
 array([       0.65]),
 array([    0.72166]),
 array([[    0.64617]]),
 array([0]))

In [43]:
ap_results=ap_per_class(tp,
                 conf,
                 pred_cls,
                 target_cls,
                 save_dir='/home/ec2-user/SageMaker/trial2',
                 eps=1e-16,
                 prefix='Pose')

RECALL
[[        0.5]
 [          1]]
___________________________________
PRECISION
[[          1]
 [          1]]
___________________________________
RECALL
[[        0.5]
 [          1]]
___________________________________
PRECISION
[[          1]
 [          1]]
___________________________________
RECALL
[[        0.5]
 [          1]]
___________________________________
PRECISION
[[          1]
 [          1]]
___________________________________
RECALL
[[        0.5]
 [          1]]
___________________________________
PRECISION
[[          1]
 [          1]]
___________________________________
RECALL
[[        0.5]
 [          1]]
___________________________________
PRECISION
[[          1]
 [          1]]
___________________________________
RECALL
[[          0]
 [        0.5]]
___________________________________
PRECISION
[[          0]
 [        0.5]]
___________________________________
RECALL
[[          0]
 [        0.5]]
___________________________________
PRECISION
[[        

In [44]:
ap_results[5]

array([[      0.995],
       [      0.995],
       [      0.995],
       [      0.995],
       [      0.995],
       [      0.375],
       [      0.375],
       [      0.995],
       [          0],
       [          0],
       [      0.995],
       [      0.995],
       [      0.375],
       [          0],
       [      0.995],
       [      0.995],
       [      0.995],
       [      0.375],
       [          0],
       [          0]])

In [45]:
(np.mean(ap_results[5])*1.25)

0.7778125

In [57]:
GT_keypoints
g=[]

for i in GT_keypoints.tolist():
    for j in i:
        g.append(str([round(j[0],2),round(j[1],2)]))
print(len(g))
g

40


['[426.76, 551.29]',
 '[443.88, 552.04]',
 '[460.01, 553.42]',
 '[475.26, 550.79]',
 '[493.51, 553.54]',
 '[508.13, 551.79]',
 '[526.51, 549.67]',
 '[542.63, 552.79]',
 '[0.0, 0.0]',
 '[0.0, 0.0]',
 '[416.88, 480.5]',
 '[435.13, 478.25]',
 '[453.01, 479.75]',
 '[470.13, 479.0]',
 '[486.63, 477.88]',
 '[504.51, 477.38]',
 '[522.13, 478.5]',
 '[538.51, 480.13]',
 '[0.0, 0.0]',
 '[0.0, 0.0]',
 '[429.13, 675.71]',
 '[444.63, 674.38]',
 '[463.63, 675.88]',
 '[477.96, 676.54]',
 '[493.8, 677.88]',
 '[511.13, 677.21]',
 '[526.13, 679.71]',
 '[546.97, 678.21]',
 '[0.0, 0.0]',
 '[0.0, 0.0]',
 '[426.76, 597.34]',
 '[441.38, 598.84]',
 '[457.63, 597.71]',
 '[475.38, 599.09]',
 '[491.26, 602.84]',
 '[508.26, 601.21]',
 '[524.88, 599.71]',
 '[542.38, 599.59]',
 '[0.0, 0.0]',
 '[0.0, 0.0]']

In [58]:
prd=[]
for i in predicted_keypoints.tolist():
    for j in i:
        prd.append(str([round(j[0],2),round(j[1],2)]))
prd

['[429.3, 675.54]',
 '[447.61, 676.02]',
 '[463.54, 675.79]',
 '[482.7, 676.39]',
 '[496.46, 675.5]',
 '[516.65, 675.75]',
 '[531.67, 674.97]',
 '[548.54, 674.85]',
 '[515.77, 676.41]',
 '[487.22, 642.35]',
 '[426.32, 602.19]',
 '[443.43, 601.24]',
 '[461.71, 601.99]',
 '[480.95, 602.05]',
 '[492.04, 601.11]',
 '[513.11, 602.07]',
 '[529.01, 601.44]',
 '[547.6, 601.62]',
 '[547.46, 674.75]',
 '[533.18, 673.25]',
 '[424.46, 552.82]',
 '[442.18, 553.15]',
 '[458.65, 552.55]',
 '[478.2, 552.5]',
 '[491.3, 551.17]',
 '[512.0, 551.43]',
 '[527.24, 550.9]',
 '[543.72, 550.46]',
 '[512.53, 553.59]',
 '[472.5, 510.74]',
 '[419.3, 479.32]',
 '[437.15, 478.05]',
 '[455.76, 478.32]',
 '[475.75, 478.25]',
 '[486.52, 477.4]',
 '[508.18, 477.18]',
 '[524.01, 476.88]',
 '[541.9, 476.77]',
 '[543.92, 550.44]',
 '[529.48, 549.44]']

In [60]:
o=[]
for i in oks_array.tolist():
    for j in i:
        o.append(round(j,2))
print(len(o))
o

40


[0.97,
 0.99,
 0.99,
 0.96,
 0.96,
 0.95,
 0.99,
 0.98,
 0.0,
 0.0,
 0.98,
 0.99,
 0.97,
 0.9,
 1.0,
 0.95,
 0.98,
 0.92,
 0.0,
 0.0,
 1.0,
 0.96,
 1.0,
 0.93,
 0.96,
 0.9,
 0.84,
 0.95,
 0.0,
 0.0,
 0.92,
 0.97,
 0.89,
 0.87,
 0.99,
 0.92,
 0.93,
 0.9,
 0.0,
 0.0]

In [49]:
tp_flat=[]
for i in tp:
    tp_flat.append(i[0])
    
tp_flat  

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0]

In [64]:
import pandas as pd
Analyse_df = pd.DataFrame(
    {'GT_Keypoints': g,
     'Predicted_Keypoints': prd,
     'OKS': o,
     'True_positives (OKS Thresh:- 0.9)': tp_flat,
     
    })

In [65]:
print("GroudTruth, Prdicted Keypoints, OKS distance between them, Truepositives when OKS treshold =0.9 ")
Analyse_df

GroudTruth, Prdicted Keypoints, OKS distance between them, Truepositives when OKS treshold =0.9 


,GT_Keypoints,Predicted_Keypoints,OKS,True_positives (OKS Thresh:- 0.9)
0,"[426.76, 551.29]","[429.3, 675.54]",0.97,1
1,"[443.88, 552.04]","[447.61, 676.02]",0.99,1
2,"[460.01, 553.42]","[463.54, 675.79]",0.99,1
3,"[475.26, 550.79]","[482.7, 676.39]",0.96,1
4,"[493.51, 553.54]","[496.46, 675.5]",0.96,1
5,"[508.13, 551.79]","[516.65, 675.75]",0.95,1
6,"[526.51, 549.67]","[531.67, 674.97]",0.99,1
7,"[542.63, 552.79]","[548.54, 674.85]",0.98,1
8,"[0.0, 0.0]","[515.77, 676.41]",0.00,0
9,"[0.0, 0.0]","[487.22, 642.35]",0.00,0


In [54]:
i = np.argsort(-conf)
tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
unique_classes, nt = np.unique(target_cls, return_counts=True)
nc = unique_classes.shape[0]
nc

20